In [2]:
!pip install onnxoptimizer


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: c:\users\пользователь\ml_models\ml_env\scripts\python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime as ort
import numpy as np
from onnxruntime.quantization import quantize_dynamic, QuantType
import onnxoptimizer
import os
from onnxruntime.transformers.optimizer import optimize_model

# Задание 1

In [4]:
# Определяем простую нейросеть
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 5)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Создаём модель и инициализируем случайные веса
model = SimpleNN()
model.eval()

SimpleNN(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
)

In [6]:
# Создаём тестовый вход
dummy_input = torch.randn(4, 10)

In [7]:
# Экспорт в ONNX с динамическими осями
onnx_path = "model.onnx"
dynamic_axes = {'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}

In [8]:
torch.onnx.export(model, dummy_input, onnx_path, 
                  input_names=['input'], output_names=['output'],
                  dynamic_axes=dynamic_axes, opset_version=11)

In [9]:
# Проверяем корректность: загружаем модель ONNX и сравниваем предсказания
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

In [10]:
# Создаём сессию ONNX Runtime
ort_session = ort.InferenceSession(onnx_path)

In [11]:
# Преобразуем входные данные в формат numpy
input_numpy = dummy_input.numpy()
onnx_inputs = {ort_session.get_inputs()[0].name: input_numpy}
onnx_outputs = ort_session.run(None, onnx_inputs)

In [12]:
# Сравниваем с PyTorch
pytorch_outputs = model(dummy_input).detach().numpy()
print("Разница между предсказаниями PyTorch и ONNX:", np.max(np.abs(pytorch_outputs - onnx_outputs[0])))

Разница между предсказаниями PyTorch и ONNX: 4.4703484e-08


# Задание 2

In [13]:
# Квантование модели в ONNX Runtime
quantized_model_path = "model_quantized.onnx"
quantize_dynamic(onnx_path, quantized_model_path, weight_type=QuantType.QInt8)

In [14]:
# Определяем разницу в размере между исходной и квантованной моделью
original_size = os.path.getsize(onnx_path) / 1024  # в КБ
quantized_size = os.path.getsize(quantized_model_path) / 1024  # в КБ
print(f"Размер исходной модели: {original_size:.2f} KB")
print(f"Размер квантованной модели: {quantized_size:.2f} KB")

Размер исходной модели: 1.73 KB
Размер квантованной модели: 2.51 KB


In [15]:
# Проверяем предсказания квантованной модели
ort_session_quantized = ort.InferenceSession(quantized_model_path)
quantized_outputs = ort_session_quantized.run(None, onnx_inputs)

In [16]:
# Оценка разницы в предсказаниях
print("Разница между предсказаниями ONNX и квантованной ONNX:", np.max(np.abs(onnx_outputs[0] - quantized_outputs[0])))

Разница между предсказаниями ONNX и квантованной ONNX: 0.23362696


# Задание 3

In [17]:
# Оптимизация модели в ONNX с использованием opt_level=1
optimized_model_path = "model_optimized.onnx"
optimized_model = optimize_model(onnx_path, opt_level=1)
optimized_model.save_model_to_file(optimized_model_path)

In [18]:
# Проверяем предсказания оптимизированной модели
ort_session_optimized = ort.InferenceSession(optimized_model_path)
optimized_outputs = ort_session_optimized.run(None, onnx_inputs)

# Оценка разницы в предсказаниях
print("Разница между предсказаниями ONNX и оптимизированной ONNX:", np.max(np.abs(onnx_outputs[0] - optimized_outputs[0])))

Разница между предсказаниями ONNX и оптимизированной ONNX: 0.0


In [19]:
# Определяем разницу в размере между исходной и квантованной моделью
original_size = os.path.getsize(onnx_path) / 1024  # в КБ
quantized_size = os.path.getsize(quantized_model_path) / 1024  # в КБ
optimized_size = os.path.getsize(optimized_model_path) / 1024  # в КБ
print(f"Размер исходной модели: {original_size:.2f} KB")
print(f"Размер квантованной модели: {quantized_size:.2f} KB")
print(f"Размер оптимизированной модели: {optimized_size:.2f} KB")

Размер исходной модели: 1.73 KB
Размер квантованной модели: 2.51 KB
Размер оптимизированной модели: 1.91 KB
